### Import Libraries


In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

### Load Data


In [2]:
df = pd.read_csv("../data/spacex_api_labelled.csv")
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


### Mark Launch Sites


In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = df[["LaunchSite", "Latitude", "Longitude", "Class"]]
launch_sites_df = spacex_df.groupby(["LaunchSite"], as_index=False).first()
launch_sites_df = launch_sites_df[["LaunchSite", "Latitude", "Longitude"]]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [5]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    nasa_coordinate, radius=1000, color="#d35400", fill=True
).add_child(folium.Popup("NASA Johnson Space Center"))

# Create a circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "NASA JSC",
    ),
)

site_map.add_child(circle)
site_map.add_child(marker)

In [6]:
site_lat = launch_sites_df["Latitude"]
print(type(site_lat))

<class 'pandas.core.series.Series'>


In [7]:
# Add a location marker for each launch site
site_lat = launch_sites_df["Latitude"]
site_long = launch_sites_df["Longitude"]

for i in site_lat.index:
    coord = [site_lat[i], site_long[i]]
    circle = folium.Circle(coord, radius=1000, color="#d35400", fill=True).add_child(
        folium.Popup(launch_sites_df["LaunchSite"][i])
    )
    marker = folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>'
            % launch_sites_df["LaunchSite"][i],
        ),
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

In [8]:
# Mark the success / failed launches for each site
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

df["marker_color"] = df["Class"].apply(lambda x: "green" if x == 1 else "red")

for index, record in df.iterrows():
    marker = folium.Marker(
        [record["Latitude"], record["Longitude"]],
        icon=folium.Icon(color=record["marker_color"]),
    )
    marker_cluster.add_child(marker)

site_map

In [9]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position="topright",
    separator=" Long: ",
    empty_string="NaN",
    lng_first=False,
    num_digits=20,
    prefix="Lat:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

### Mark Distances


In [10]:
from math import sin, cos, sqrt, atan2, radians


# Calculate the distances between a launch site to its proximities
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Approximate radius of earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
# Coordenates
launch_site_c = [28.56186, -80.57738]
coastline_c = [28.56291, -80.56794]
railway_c = [28.56301, -80.57683]
highway_c = [28.56341, -80.57678]
city_c = [28.55986, -80.79892]

distance_coastline = calculate_distance(launch_site_c[0], launch_site_c[1], coastline_c[0], coastline_c[1])
distance_railway = calculate_distance(launch_site_c[0], launch_site_c[1], railway_c[0], railway_c[1])
distance_highway = calculate_distance(launch_site_c[0], launch_site_c[1], highway_c[0], highway_c[1])
distance_city = calculate_distance(launch_site_c[0], launch_site_c[1], city_c[0], city_c[1])


In [12]:
# Create a marker with distance to a closest city, railway, highway, coastline, etc.
# Draw a line between the marker to the launch site

marker_coastline = folium.Marker(
    coastline_c,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 30; color:#d35400;"><b>%s</b></div>'
        % "{:10.2f} KM".format(round(distance_coastline, 2)),
    ),
)

marker_railway = folium.Marker(
    railway_c,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 30; color:#d35400;"><b>%s</b></div>'
        % "{:10.2f} KM".format(round(distance_railway, 2)),
    ),
)

marker_highway = folium.Marker(
    highway_c,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 30; color:#d35400;"><b>%s</b></div>'
        % "{:10.2f} KM".format(round(distance_highway, 2)),
    ),
)

marker_city = folium.Marker(
    city_c,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 30; color:#d35400;"><b>%s</b></div>'
        % "{:10.2f} KM".format(round(distance_city, 2)),
    ),
)

marker_cluster.add_child(marker_coastline)
marker_cluster.add_child(marker_railway)
marker_cluster.add_child(marker_highway)
marker_cluster.add_child(marker_city)

line_coastline = folium.PolyLine(locations=[launch_site_c, coastline_c], weight = 1)
line_railway = folium.PolyLine(locations=[launch_site_c, railway_c], weight = 1)
line_highway = folium.PolyLine(locations=[launch_site_c, highway_c], weight = 1)
line_city = folium.PolyLine(locations=[launch_site_c, city_c], weight = 1)

site_map.add_child(line_coastline)
site_map.add_child(line_railway)
site_map.add_child(line_highway)
site_map.add_child(line_city)

site_map

### Save Map

In [13]:
folium.TileLayer('cartodbdark_matter').add_to(site_map)
site_map.save("../src/assets/launch_sites_map.html")